In [14]:
## Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import sqlite3    ## SQL Interface
import pickle     ## Used to save your data - Converts objects to byte stream and vice versa

## Modules to perform Text Preprocessing
import re
import nltk
from nltk.stem.snowball import SnowballStemmer

In [6]:
with open('student_info2','rb') as pickle_file:
    data = pickle.load(pickle_file)
data.head()

,SEM 1 SGPA,SEM 1 KT,SEM 2 SGPA,SEM 2 KT,SEM 3 SGPA,SEM 3 KT,SEM 4 SGPA,SEM 4 KT,SEM 5 SGPA,SEM 5 KT,...,Mode_Of_Transportation,2_hrs_lect,Submissions,5_hrs_lect,5_hrs_pracs,Coaching_classes,Teacher's Feedback,Label,Scocial_Skills,Average pointer
0,7.93,0,7.93,0,7.38,0.0,7.30,0.0,7.40,0,...,1,0,1,1,0,0,Good student,1,1,7.58000
1,7.75,0,7.78,0,7.68,0.0,8.93,0.0,8.33,0,...,1,0,1,1,1,0,Good leadership skills,1,0,8.24625
2,6.15,0,6.09,0,6.00,0.0,7.10,0.0,6.10,0,...,3,1,1,1,1,0,Hard working,1,0,6.39250
3,5.75,0,6.54,0,5.90,0.0,5.98,0.0,6.00,0,...,1,0,0,0,1,1,Responsible,1,0,6.18375
4,5.67,0,6.81,0,6.60,2.0,6.50,0.0,6.70,0,...,1,0,1,1,1,0,Good leadership skills,1,1,6.27250


In [7]:
data.columns

Index(['SEM 1 SGPA', 'SEM 1 KT', 'SEM 2 SGPA', 'SEM 2 KT', 'SEM 3 SGPA',
       'SEM 3 KT', 'SEM 4 SGPA', 'SEM 4 KT', 'SEM 5 SGPA', 'SEM 5 KT',
       'SEM 6 SGPA', 'SEM 6 KT', 'SEM 7 SGPA', 'SEM 7 KT', 'SEM 8 SGPA',
       'Hours_On_Assignment', 'Hours_On_Studies', 'Travel_Time', 'Attendance',
       'Internet_Availability', 'Internet_Speed', 'Mode_Of_Transportation',
       '2_hrs_lect', 'Submissions', '5_hrs_lect', '5_hrs_pracs',
       'Coaching_classes', 'Teacher's Feedback', 'Label', 'Scocial_Skills',
       'Average pointer'],
      dtype='object')

In [9]:
final = data.sample(frac=1,random_state = 123).reset_index(drop=True)  #Shuffling the DataFrame.

In [10]:
final['Label'].value_counts()

1    162
0    100
Name: Label, dtype: int64

In [11]:
conn = sqlite3.connect('final.sqlite')    #Saving the changes made above into a new sqlite file
c=conn.cursor()
final.to_sql('Reviews', conn, if_exists='replace')
conn.close()

C:\Users\Pratheesh\Anaconda3\lib\site-packages\pandas\core\generic.py:1362: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


In [12]:
conn = sqlite3.connect('final.sqlite')  #Loading the sqlite file for future use
final = pd.read_sql_query("""SELECT * FROM Reviews""", conn)
conn.close()
final.head()

,index,SEM 1 SGPA,SEM 1 KT,SEM 2 SGPA,SEM 2 KT,SEM 3 SGPA,SEM 3 KT,SEM 4 SGPA,SEM 4 KT,SEM 5 SGPA,...,Mode_Of_Transportation,2_hrs_lect,Submissions,5_hrs_lect,5_hrs_pracs,Coaching_classes,Teacher's Feedback,Label,Scocial_Skills,Average pointer
0,0,7.10,0,6.85,0,7.20,1.0,7.30,0.0,7.60,...,1,0,0,0,0,0,Disciplined and hard working,1,0,7.13125
1,1,7.20,0,7.10,0,6.11,0.0,6.67,0.0,7.14,...,1,1,1,0,0,1,Good leadership skills,1,1,7.05375
2,2,3.80,4,4.20,2,4.20,1.0,3.80,2.0,4.35,...,1,0,1,0,0,0,Very talkative and Poor attendance,0,0,4.05875
3,3,6.91,1,6.60,2,5.82,1.0,6.10,0.0,7.71,...,1,0,1,0,1,0,Respectful to Authority and Others,1,0,6.50500
4,4,3.50,1,3.40,2,4.00,2.0,4.10,2.0,3.63,...,1,0,0,0,0,0,Bunk lectuer,0,0,3.81500


In [15]:
def cleanpunc(sentence):
    '''This function cleans all the punctuation or special characters from a given sentence'''
    cleaned = re.sub(r'[?|@|!|^|%|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned

In [17]:
def preprocessing(series):
    '''The function takes a Pandas Series object containing text in all the cells
       And performs following Preprocessing steps on each cell:
       1. Clean text from punctuations and special characters
       2. Retain only non-numeric Latin characters.
       3. Apply stemming to all the words in the sentence
       
       Return values:
       1. final_string - List of cleaned sentences
       2. list_of_sent - List of lists which can be used as input to the W2V model'''
    
    i = 0
    str1=" "
    final_string = []    ## This list will contain cleaned sentences
    list_of_sent = []    ## This is a list of lists used as input to the W2V model at a later stage
    sno = SnowballStemmer("english")
    
    for sent in series.values:
        ## 
        filtered_sent = []
        sent = cleanpunc(sent)    ## Clean the punctuations and special characters
        ## Sentences are cleaned and words are handled individually
        for cleaned_words in sent.split():
            ## Only consider non-numeric words with length at least 3
            if((cleaned_words.isalpha())):
                ## Only consider words which are not stopwords and convert them to lowet case
                if(cleaned_words.lower() not in stop):
                    ## Apply snowball stemmer and add them to the filtered_sent list
                    s = (sno.stem(cleaned_words.lower()))#.encode('utf-8')
                    filtered_sent.append(s)    ## This contains all the cleaned words for a sentence
                    
        ## Below list is a list of lists used as input to W2V model later
        list_of_sent.append(filtered_sent)
        ## Join back all the words belonging to the same sentence
        str1 = " ".join(filtered_sent)
        ## Finally add the cleaned sentence in the below list
        final_string.append(str1)
        #print(i)
        i += 1
    return final_string, list_of_sent

In [18]:
final_string, list_of_sent = preprocessing(final["Teacher's Feedback"])  #Preprocessing the 'Text' column in the DataFrame

In [19]:
final['Cleaned_Feedback']=final_string   #Creating a new column of Preprocessed Stemmed text

In [20]:
final.head()

,index,SEM 1 SGPA,SEM 1 KT,SEM 2 SGPA,SEM 2 KT,SEM 3 SGPA,SEM 3 KT,SEM 4 SGPA,SEM 4 KT,SEM 5 SGPA,...,2_hrs_lect,Submissions,5_hrs_lect,5_hrs_pracs,Coaching_classes,Teacher's Feedback,Label,Scocial_Skills,Average pointer,Cleaned_Feedback
0,0,7.10,0,6.85,0,7.20,1.0,7.30,0.0,7.60,...,0,0,0,0,0,Disciplined and hard working,1,0,7.13125,disciplin hard work
1,1,7.20,0,7.10,0,6.11,0.0,6.67,0.0,7.14,...,1,1,0,0,1,Good leadership skills,1,1,7.05375,good leadership skill
2,2,3.80,4,4.20,2,4.20,1.0,3.80,2.0,4.35,...,0,1,0,0,0,Very talkative and Poor attendance,0,0,4.05875,talkat poor attend
3,3,6.91,1,6.60,2,5.82,1.0,6.10,0.0,7.71,...,0,1,0,1,0,Respectful to Authority and Others,1,0,6.50500,respect author other
4,4,3.50,1,3.40,2,4.00,2.0,4.10,2.0,3.63,...,0,0,0,0,0,Bunk lectuer,0,0,3.81500,bunk lectuer


In [21]:
conn = sqlite3.connect('final.sqlite')   #Saving the changes
c=conn.cursor()
final.to_sql('Reviews', conn, if_exists='replace', index = False)
conn.close()

C:\Users\Pratheesh\Anaconda3\lib\site-packages\pandas\core\generic.py:1362: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


In [22]:
with open('list_of_sent_for_input_to_w2v.pkl', 'wb') as pickle_file:  #Saving the list of sentences in a pickle file for future use
    pickle.dump(list_of_sent, pickle_file)